In [5]:
import tkinter as tk
from tkinter import messagebox
from typing import Optional, Tuple, List, Set
import sys
#from PIL import Image, ImageTk
#import os
from chess_engine import ChessEngine, Piece, Board

class ImprovedChessUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Chess Game vs AI")
        
        # Initialize the chess engine
        self.engine = ChessEngine()
        
        # Game state
        self.selected_square: Optional[Tuple[int, int]] = None
        self.valid_moves: Set[Tuple[int, int]] = set()
        self.player_color = 'white'
        self.current_turn = 'white'
        self.board_flipped = True
        
        # Colors
        self.LIGHT_SQUARE = '#F0D9B5'
        self.DARK_SQUARE = '#B58863'
        self.SELECTED_COLOR = '#646D40'
        self.VALID_MOVE_COLOR = '#AAB648'
        self.LAST_MOVE_COLOR = '#CDD26A'
        
        self.last_move: Optional[Tuple[Tuple[int, int], Tuple[int, int]]] = None
        
        # Load piece images
        self.piece_images = {}
        #self.load_piece_images()
        self.piece_images = self.load_piece_images()
        
        self.setup_ui()
     
    #def load_piece_images(self):
    def load_piece_images(self) -> dict:

        """Load piece images for the game."""
        #images to load
        images = {

            'white_king': tk.PhotoImage(file='images/wk.png'),

            'black_king': tk.PhotoImage(file='images/bk.png'),

            'white_queen': tk.PhotoImage(file='images/wq.png'),

            'black_queen': tk.PhotoImage(file='images/bq.png'),

            'white_rook': tk.PhotoImage(file='images/wr.png'),

            'black_rook': tk.PhotoImage(file='images/br.png'),

            'white_bishop': tk.PhotoImage(file='images/wb.png'),

            'black_bishop': tk.PhotoImage(file='images/bb.png'),

            'white_knight': tk.PhotoImage(file='images/wn.png'),

            'black_knight': tk.PhotoImage(file='images/bn.png'),

            'white_pawn': tk.PhotoImage(file='images/wp.png'),

            'black_pawn': tk.PhotoImage(file='images/bp.png'),
            'zz': tk.PhotoImage(file='images/zz.png'),

        }

        return images
        
    def setup_ui(self):
        # Create main container
        self.container = tk.Frame(self.root)
        self.container.pack(expand=True, fill='both', padx=20, pady=20)
        
        # Create left panel for the board
        self.board_frame = tk.Frame(self.container)
        self.board_frame.pack(side=tk.LEFT, padx=10)
        
        # Create right panel for controls
        self.control_panel = tk.Frame(self.container)
        self.control_panel.pack(side=tk.RIGHT, padx=10)
        
        # Initialize the board squares
        self.squares = [[None for _ in range(8)] for _ in range(8)]
        self.square_labels = [[None for _ in range(8)] for _ in range(8)]
        self.create_board()
        
        # Add controls
        self.create_control_panel()
        #self.flip_board()
        # Initial board update
        
        self.update_board()
        
    def create_board(self):
        # Create rank labels (1-8)
        for i in range(8):
            rank_label = tk.Label(self.board_frame, text=str(8-i), width=2)
            rank_label.grid(row=i, column=0, padx=(0, 5))
        
        # Create file labels (a-h)
        for i in range(8):
            file_label = tk.Label(self.board_frame, text=chr(97 + i))
            file_label.grid(row=8, column=i+1)
        
        # Create board squares
        for row in range(8):
            for col in range(8):
                square = tk.Frame(
                    self.board_frame,
                    width=150,
                    height=150,
                    bg=self.get_square_color(row, col)
                )
                square.grid(row=row, column=col+1)
                square.grid_propagate(False)
                
                # Create label for the piece image
                label = tk.Label(
                    square,
                    bd=0,
                    highlightthickness=0,
                    bg=self.get_square_color(row, col)
                )
                label.pack(expand=True, fill='both')
                label.bind('<Button-1>', lambda e, r=row, c=col: self.square_clicked(r, c))
                
                self.squares[row][col] = square
                self.square_labels[row][col] = label
                
    def create_control_panel(self):
        # Turn indicator
        self.turn_label = tk.Label(
            self.control_panel,
            text="Turn: White",#Older Code ---
            font=('Arial', 14, 'bold'),
            pady=10
        )
        self.turn_label.pack()
        
        # Control buttons
        tk.Button(
            self.control_panel,
            text="New Game",
            command=self.new_game,
            width=20,
            pady=5
        ).pack(pady=5)
        
        tk.Button(
            self.control_panel,
            text="Flip Board",
            command=self.flip_board,
            width=20,
            pady=5
        ).pack(pady=5)
        
        # Status messages
        self.status_label = tk.Label(
            self.control_panel,
            text="",#Older Code --- 
            font=('Arial', 12),
            wraplength=200,
            pady=10
        )
        self.status_label.pack()
    
    def get_square_color(self, row: int, col: int) -> str:
        return self.LIGHT_SQUARE if (row + col) % 2 == 0 else self.DARK_SQUARE
    
    def get_piece_image(self, piece: Piece) -> Optional[tk.PhotoImage]:

        if not piece:

            key = f"zz" # Empty square image
        else:
            key = f"{piece.color}_{piece.type}"

        return self.piece_images.get(key)
    
    def square_clicked(self, row: int, col: int):
        if self.current_turn != self.player_color:
            return
        
        clicked_pos = (row, col)
        piece = self.engine.board.get_piece(row, col)
        
        # If no square is selected
        if self.selected_square is None:
            if piece and piece.color == self.player_color:
                self.selected_square = clicked_pos
                self.valid_moves = set(self.engine.board.get_valid_moves(row, col))
                self.update_board_highlights()
        
        # If a square is already selected
        else:
            # If clicking on a valid move square
            if clicked_pos in self.valid_moves:
                self.make_move(self.selected_square, clicked_pos)
                self.selected_square = None
                self.valid_moves.clear()
                self.current_turn = 'black'
                self.update_turn_indicator()
                self.update_board_highlights()
                self.root.after(500, self.make_ai_move)
            
            # If clicking on a different piece of same color
            elif piece and piece.color == self.player_color:
                self.selected_square = clicked_pos
                self.valid_moves = set(self.engine.board.get_valid_moves(row, col))
                self.update_board_highlights()
            
            # If clicking on an invalid square
            else:
                self.selected_square = None
                self.valid_moves.clear()
                self.update_board_highlights()
    
    def make_move(self, start: Tuple[int, int], end: Tuple[int, int]):
        self.engine.board.make_move(start, end)
        self.last_move = (start, end)
        self.update_board()
    
    def make_ai_move(self):
        best_move = self.engine.get_best_move('black', depth=3)
        if best_move:
            self.make_move(best_move[0], best_move[1])
            self.current_turn = 'white'
            self.update_turn_indicator()
    
    def update_board(self):
        for row in range(8):
            for col in range(8):
                self.update_square(row, col)
        
        self.update_board_highlights()
    
    def update_square(self, row: int, col: int):
        label = self.square_labels[row][col]
        piece = self.engine.board.get_piece(row, col)
        
        # Update piece image
        if piece:
            piece_image = self.get_piece_image(piece)
            label.configure(image=piece_image)
            label.image = piece_image
          # Keep a reference to prevent garbage collection
        else:
            piece_image = self.get_piece_image(piece)
            label.configure(image=piece_image)
            label.image = piece_image
            
    
    def update_board_highlights(self):
        for row in range(8):
            for col in range(8):
                square = self.squares[row][col]
                label = self.square_labels[row][col]
                base_color = self.get_square_color(row, col)
                
                # Selected square
                if (row, col) == self.selected_square:
                    square.configure(bg=self.SELECTED_COLOR)
                    label.configure(bg=self.SELECTED_COLOR)
                
                # Valid move squares
                elif (row, col) in self.valid_moves:
                    square.configure(bg=self.VALID_MOVE_COLOR)
                    label.configure(bg=self.VALID_MOVE_COLOR)
                
                # Last move squares
                elif self.last_move and (row, col) in self.last_move:
                    square.configure(bg=self.LAST_MOVE_COLOR)
                    label.configure(bg=self.LAST_MOVE_COLOR)
                
                # Default square color
                else:
                    square.configure(bg=base_color)
                    label.configure(bg=base_color)
    
    def update_turn_indicator(self):
        self.turn_label.configure(text=f"Turn: {'White' if self.current_turn == 'white' else 'Black'}")
    
    def new_game(self):
        self.engine = ChessEngine()
        self.selected_square = None
        self.valid_moves.clear()
        self.current_turn = 'white'
        self.last_move = None
        self.update_turn_indicator()
        self.update_board()
    
    def flip_board(self):
        """
        Flips the chess board view 180 degrees and updates the player's perspective.
        This allows for playing from either white's or black's viewpoint.
        """
        # Toggle player's perspective
        self.player_color = 'black' if self.player_color == 'white' else 'white'
        
        # Store current board state
        old_squares = [[square for square in row] for row in self.squares]
        
        # Rearrange squares in the grid
        for row in range(8):
            for col in range(8):
                # Calculate new positions
                new_row = 7 - row
                new_col = 7 - col
                
                # Update grid position
                old_squares[row][col].grid(row=row, column=col)
                
                # Update internal squares array
                self.squares[row][col] = old_squares[new_row][new_col]
        
        # Update the board display
        self.update_board()
        
        # Clear any selections and highlights
        self.selected_square = None
        self.unhighlight_all()
        
        # Update the button text to show current perspective
        self.flip_board_btn.config(
            text=f"Flip Board (Current: {self.player_color.capitalize()})"
        )

def main():
    root = tk.Tk()
    app = ImprovedChessUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()